In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean

In [2]:
data = pd.read_csv('training.csv')
validate = pd.read_csv('validation.csv')
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Primary_1994_06_07,General_1994_11_08,Primary_1996_03_26,General_1996_11_05,Primary_1998_06_02,General_1998_11_03,Primary_2000_03_07,General_2000_11_07,Primary_2002_03_05,General_2002_11_05,...,Education,Housing_Type,Language,Registered_Party,Race,gender,place_of_birth,Registration_Date,uid,year_of_birth
0,NaN,NaN,True,True,True,True,True,True,True,True,...,unknown,multi_family,English,Democratic,5_other,M,us_born,1996-02-25,28f9b8c0a8ef9f80deaae8174efe7e9c03b4546e,1942.0
1,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,False,...,unknown,multi_family,English,Democratic,2_latino,F,foreign_born,1999-03-22,52c90adf941dda65cce99e5dddcfb4217d3fbffd,1935.0
2,False,False,False,False,False,False,False,True,False,False,...,2_college,multi_family,English,Republican,1_white,M,foreign_born,1992-08-25,6d90c80ed6264f06077023799e9b043de0c32f70,1974.0
3,False,False,False,False,False,False,True,True,False,False,...,2_college,single_family,Spanish,Republican,2_latino,F,us_born,1988-09-30,85444bd571cea266e2b43aea8e90dcba06cb2ebe,1970.0
4,False,False,True,True,True,True,True,False,False,False,...,1_post_grad,single_family,English,Republican,5_other,M,foreign_born,1986-03-22,b5def636d3bbc0168539a640c76eb372a5f7f1ca,1934.0


In [3]:
# Convert T/F data to binary and make a list of election result columns

def tf_to_bin(df):
    columns = list(data)
    election_data_cols = []
    new_df = df.copy()
    for column in columns:
        if 'Primary' in column or 'General' in column:
            new_df[column] = (new_df[column]*1)
            election_data_cols.append(column)
    return new_df


data = tf_to_bin(data)
data.head()

,Primary_1994_06_07,General_1994_11_08,Primary_1996_03_26,General_1996_11_05,Primary_1998_06_02,General_1998_11_03,Primary_2000_03_07,General_2000_11_07,Primary_2002_03_05,General_2002_11_05,...,Education,Housing_Type,Language,Registered_Party,Race,gender,place_of_birth,Registration_Date,uid,year_of_birth
0,NaN,NaN,1,1,1,1,1,1,1,1,...,unknown,multi_family,English,Democratic,5_other,M,us_born,1996-02-25,28f9b8c0a8ef9f80deaae8174efe7e9c03b4546e,1942.0
1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,...,unknown,multi_family,English,Democratic,2_latino,F,foreign_born,1999-03-22,52c90adf941dda65cce99e5dddcfb4217d3fbffd,1935.0
2,0,0,0,0,0,0,0,1,0,0,...,2_college,multi_family,English,Republican,1_white,M,foreign_born,1992-08-25,6d90c80ed6264f06077023799e9b043de0c32f70,1974.0
3,0,0,0,0,0,0,1,1,0,0,...,2_college,single_family,Spanish,Republican,2_latino,F,us_born,1988-09-30,85444bd571cea266e2b43aea8e90dcba06cb2ebe,1970.0
4,0,0,1,1,1,1,1,0,0,0,...,1_post_grad,single_family,English,Republican,5_other,M,foreign_born,1986-03-22,b5def636d3bbc0168539a640c76eb372a5f7f1ca,1934.0


In [4]:
# Get percent of voters that turnout for the general after voting in the primary for later weighting
primary_recall = []
non_voters = []
def primary_to_general(df):
    colnames = list(df)
    year = colnames[0][8:12]
    df_new = df.copy()
    df_new = df_new.dropna()
    y_pred = df_new.iloc[:,0]
    y_true = df_new.iloc[:,1]
    tn, fp, fn, tp = confusion_matrix(y_true.tolist(), y_pred.tolist()).ravel()
    primary_recall.append(tp/(tp+fp))
    non_voters.append(tn/(tn+fn))
    print(f"pct of primary voters who also voted in the general in {year} "+str(tp/(tp+fp)))
    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    

midterm_2014 = data[['Primary_2014_06_03','General_2014_11_04']]
midterm_2010 = data[['Primary_2010_06_08','General_2010_11_02']]
midterm_2006 = data[['Primary_2006_06_06','General_2006_11_07']]
midterm_2002 = data[['Primary_2002_03_05','General_2002_11_05']]
midterm_1998 = data[['Primary_1998_06_02','General_1998_11_03']]
midterm_1994 = data[['Primary_1994_06_07','General_1994_11_08']]
midterm_list=[midterm_2014,midterm_2010,midterm_2006,midterm_2002,midterm_1998,midterm_1994]
for midterm in midterm_list:
    primary_to_general(midterm)
    
avg_primary_conversion = mean(primary_recall)
print()
print('Average conversion of primary vote to general vote in midterms: '+str(avg_primary_conversion))

pct of primary voters who also voted in the general in 2014 0.8979616157847831
pct of primary voters who also voted in the general in 2010 0.9350510146877452
pct of primary voters who also voted in the general in 2006 0.9191619583082006
pct of primary voters who also voted in the general in 2002 0.8426497703577056
pct of primary voters who also voted in the general in 1998 0.8820692513469726
pct of primary voters who also voted in the general in 1994 0.9366195779951783

Average conversion of primary vote to general vote in midterms: 0.9022521980800975


#### Get baselines for the ml model to beat

In [5]:
mean(non_voters)

0.6169528732294152

#### Conversion of categorical variables to dummy variables
and other data cleaning

In [6]:
def clean_data(df):
    data = df.copy()
    cat_list= ['Education','Housing_Type','Language','Registered_Party','Race','gender','place_of_birth']
    for column in cat_list:
        dummy_col = pd.get_dummies(data[column],drop_first=True)
        dummy_col_list = list(dummy_col)
        new_col_list = []
        for col_name in dummy_col_list:
            col_name = col_name.lower()
            col_name = col_name.replace('unknown',str(column)+'_unknown')
            new_col_list.append(col_name)
        dummy_col.columns=new_col_list
        data.drop([column],axis=1,inplace=True)
        data = pd.concat([data,dummy_col],axis=1)
    pre_2006 = ['Primary_1994_06_07','General_1994_11_08','Primary_1996_03_26','General_1996_11_05','Primary_1998_06_02','General_1998_11_03','Primary_2000_03_07','General_2000_11_07','Primary_2002_03_05','General_2002_11_05','Primary_2004_03_02','General_2004_11_02','Primary_2006_06_06','General_2006_11_07']
    for column in pre_2006:
        data[column] = data[column].fillna(0)
    data['year_of_birth']=data['year_of_birth'].fillna(data['year_of_birth'].median())
    droplist = ['Presidential_Primary_2008_02_05','Primary_2008_06_03','General_2008_11_04','Primary_2010_06_08','General_2010_11_02','Primary_2012_06_05','General_2012_11_06','Primary_2014_06_03','General_2014_11_04','Primary_2016_06_07','General_2016_11_08','Primary_2018_06_05','Registration_Date','Primary_1994_06_07','General_1994_11_08']
    for column in droplist:
        data = data.drop(column,axis=1)
    uid = data['uid']
    data = data.drop('uid',axis=1)
    return data

#### Train and validate a logistic regression model on the data
Lacking a true response, I have chosen to use the 2006 midterms as the response variable. It presented the most similar conditions to the 2018 midterms - an unpopular republican president in a midterm where opposition party turnout was expected to be high (a "wave year")

In [7]:
learn_data = clean_data(data)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(learn_data.drop('General_2006_11_07',axis=1), 
                                                    learn_data['General_2006_11_07'], test_size=0.30, 
                                                    random_state=66)

In [9]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90    190470
           1       0.84      0.75      0.79     95700

    accuracy                           0.87    286170
   macro avg       0.86      0.84      0.85    286170
weighted avg       0.87      0.87      0.87    286170



#### Validate the model

In [11]:
# gut check
y_test_list = list(y_test)
pred_list = list(predictions)
actuals_vs_pred = zip(pred_list,y_test_list)

y_test_df = pd.DataFrame(actuals_vs_pred, index = y_test.index, columns=(['predicted','actuals']))
y_test_df = y_test_df.merge(learn_data['General_2006_11_07'], how='inner', left_index = True, right_index = True)
y_test_df.sample(10)

,predicted,actuals,General_2006_11_07
733459,0,0,0
850194,0,0,0
338217,0,0,0
851336,0,0,0
607515,0,0,0
185836,0,0,0
462619,0,0,0
69950,0,0,0
155876,1,1,1
372941,1,1,1


In [12]:
validate = tf_to_bin(validate)
validate_data = clean_data(validate)
validate_preds = logmodel.predict(validate_data.drop('General_2006_11_07',axis=1))
validate_actuals = validate_data['General_2006_11_07']

In [13]:
print(classification_report(validate_actuals,validate_preds))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90    125907
           1       0.84      0.75      0.79     63692

    accuracy                           0.87    189599
   macro avg       0.86      0.84      0.85    189599
weighted avg       0.87      0.87      0.87    189599



#### Clean final data

In [14]:
# would have been better to modify cleaning function to handle this, but the time:improvement ratio wasn't good
col_names = list(learn_data)
col_names.remove('General_2006_11_07')
drop_list = ['Primary_1994_06_07','General_1994_11_08','Primary_1996_03_26','General_1996_11_05','Primary_1998_06_02','General_1998_11_03','Primary_2000_03_07','General_2000_11_07','Primary_2002_03_05','General_2002_11_05','Primary_2004_03_02','General_2004_11_02','Primary_2006_06_06','General_2006_11_07','Presidential_Primary_2008_02_05','Registration_Date']
test_df = pd.read_csv('validation.csv')

cat_list= ['Education','Housing_Type','Language','Registered_Party','Race','gender','place_of_birth']
for column in cat_list:
    dummy_col = pd.get_dummies(test_df[column],drop_first=True)
    dummy_col_list = list(dummy_col)
    new_col_list = []
    for col_name in dummy_col_list:
        col_name = col_name.lower()
        col_name = col_name.replace('unknown',str(column)+'_unknown')
        new_col_list.append(col_name)
    dummy_col.columns=new_col_list
    test_df.drop([column],axis=1,inplace=True)
    test_df = pd.concat([test_df,dummy_col],axis=1)

for column in drop_list:
    test_df = test_df.drop(column,axis=1)

real_uid = test_df['uid']
test_df = test_df.drop('uid',axis=1)
test_df['year_of_birth']=test_df['year_of_birth'].fillna(data['year_of_birth'].median())

print(col_names)
print(list(test_df))
    
test_df.columns = col_names

pre_2006 = ['Primary_1996_03_26','General_1996_11_05','Primary_1998_06_02','General_1998_11_03','Primary_2000_03_07','General_2000_11_07','Primary_2002_03_05','General_2002_11_05','Primary_2004_03_02','General_2004_11_02','Primary_2006_06_06']
for column in pre_2006:
    test_df[column] = test_df[column].fillna(0)

['Primary_1996_03_26', 'General_1996_11_05', 'Primary_1998_06_02', 'General_1998_11_03', 'Primary_2000_03_07', 'General_2000_11_07', 'Primary_2002_03_05', 'General_2002_11_05', 'Primary_2004_03_02', 'General_2004_11_02', 'Primary_2006_06_06', 'year_of_birth', '2_college', '3_high_school', 'Education_unknown', 'single_family', 'spanish', 'democratic', 'green', 'libertarian', 'natural law', 'non-partisan', 'other', 'peace and freedom', 'reform', 'republican', 'Registered_Party_unknown', '2_latino', '3_black', '4_asian', '5_other', 'Race_unknown', 'm', 'place_of_birth_unknown', 'us_born']
['Primary_2008_06_03', 'General_2008_11_04', 'Primary_2010_06_08', 'General_2010_11_02', 'Primary_2012_06_05', 'General_2012_11_06', 'Primary_2014_06_03', 'General_2014_11_04', 'Primary_2016_06_07', 'General_2016_11_08', 'Primary_2018_06_05', 'year_of_birth', '2_college', '3_high_school', 'Education_unknown', 'single_family', 'spanish', 'democratic', 'green', 'libertarian', 'natural law', 'non-partisan',

#### Generate actual predictions

In [15]:
actual_preds = logmodel.predict(test_df)

In [16]:
probabilities = logmodel.predict_proba(test_df)

In [17]:
output_df = pd.read_csv('validation.csv')
uid = output_df['uid']
uid.values

uid_list = list(uid.values)
#pred_list = list(actual_preds)
vote_pct = [x[1] for x in list(probabilities)]
#predictions = zip(uid_list,pred_list,vote_pct)
predictions = zip(uid_list,vote_pct)

output_df = pd.DataFrame(predictions, columns=(['uid','probability']))
output_df
output_df.to_csv('2018_Election_Turnout_Predictions.csv')

#### Test 2018 Primary Weight

In [18]:
test_probabilities = logmodel.predict_proba(X_test)
test_probabilities =  [x[1] for x in list(test_probabilities)]
primary_voter = list(X_train['Primary_2006_06_06'].values)
weight_test = zip(primary_voter,test_probabilities)
weight_test_df = pd.DataFrame(weight_test, columns=(['primary_voter','test_probabilities']))
weight_test_df

,primary_voter,test_probabilities
0,0,0.115916
1,0,0.040807
2,0,0.071332
3,0,0.037373
4,0,0.044214
...,...,...
286165,0,0.349489
286166,0,0.698475
286167,1,0.069911
286168,1,0.933060


In [19]:
weighted_output = []
for index, row in weight_test_df.iterrows():
    if row['primary_voter'] == 1:
        prob = 0.9*row['test_probabilities'] + 0.1*avg_primary_conversion
        weighted_output.append(prob)
    else:
        weighted_output.append(row['test_probabilities'])
test_weight = np.array([int(round(i)) for i in weighted_output])

In [20]:
print(classification_report(y_test,test_weight))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90    190470
           1       0.83      0.76      0.79     95700

    accuracy                           0.87    286170
   macro avg       0.86      0.84      0.85    286170
weighted avg       0.87      0.87      0.87    286170

